#   

# 作業目標:

    1. 藉由固定的 dataset, 來驗證不同loss function
    2. Dataset 的特性跟我們選用的loss function 對accrancy 的影響
    
    
# 作業重點: 
    請分別選用 "MSE", "binary _crossentropy"
    查看Train/test accurancy and loss rate
    

# 導入必要的函數

In [1]:
from keras.datasets import cifar10
import numpy as np
np.random.seed(10)

C:\Users\Dodo\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 資料準備

In [2]:
#取得Keras Dataset
(x_img_train,y_label_train),(x_img_test,y_label_test)=cifar10.load_data()

In [3]:
#確認 CIFAR10 Dataset 資料維度
print("train data:",'images:',x_img_train.shape,
      " labels:",y_label_train.shape) 
print("test  data:",'images:',x_img_test.shape ,
      " labels:",y_label_test.shape) 

train data: images: (50000, 32, 32, 3)  labels: (50000, 1)
test  data: images: (10000, 32, 32, 3)  labels: (10000, 1)


In [4]:
#資料正規化
x_img_train_normalize = x_img_train.astype('float32') / 255.0
x_img_test_normalize = x_img_test.astype('float32') / 255.0

In [5]:
#針對Label 做 ONE HOT ENCODE
from keras.utils import np_utils
y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)
y_label_test_OneHot.shape

(10000, 10)

# 建立模型

In [6]:
from keras.models import Sequential, clone_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

In [7]:
model = Sequential()

W0724 21:45:35.705022 60452 deprecation_wrapper.py:119] From C:\Users\Dodo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:71: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [8]:
#卷積層1

In [9]:
model.add(Conv2D(filters=32,kernel_size=(3,3),
                 input_shape=(32, 32,3), 
                 activation='relu', 
                 padding='same'))

W0724 21:45:36.006833 60452 deprecation_wrapper.py:119] From C:\Users\Dodo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:514: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0724 21:45:36.076788 60452 deprecation_wrapper.py:119] From C:\Users\Dodo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4076: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [10]:
model.add(Dropout(rate=0.25))

W0724 21:45:36.201714 60452 deprecation_wrapper.py:119] From C:\Users\Dodo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:130: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0724 21:45:36.251682 60452 deprecation.py:506] From C:\Users\Dodo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3363: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
model.add(MaxPooling2D(pool_size=(2, 2)))

W0724 21:45:36.358617 60452 deprecation_wrapper.py:119] From C:\Users\Dodo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3900: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [12]:
#卷積層2與池化層2

In [13]:
model.add(Conv2D(filters=64, kernel_size=(3, 3), 
                 activation='relu', padding='same'))

In [14]:
model.add(Dropout(0.25))

In [15]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [16]:
#建立神經網路(平坦層、隱藏層、輸出層)

In [17]:
model.add(Flatten())
model.add(Dropout(rate=0.25))

In [18]:
model.add(Dense(1024, activation='relu'))
model.add(Dropout(rate=0.25))

In [19]:
model.add(Dense(10, activation='softmax'))

In [20]:
#檢查model 的STACK
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
__________

# 載入之前訓練的模型

In [21]:
try:
    model.load_weights("SaveModel/cifarCnnModel.h5")
    print("載入模型成功!繼續訓練模型")
except :    
    print("載入模型失敗!開始訓練一個新模型")

載入模型失敗!開始訓練一個新模型


# 訓練模型

In [22]:
mse_model = clone_model(model)
mse_model.compile(loss='MSE', optimizer='sgd', metrics=['accuracy'])

bincross_model = clone_model(model)
bincross_model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])



W0724 21:45:36.978236 60452 deprecation_wrapper.py:119] From C:\Users\Dodo\Anaconda3\lib\site-packages\keras\optimizers.py:757: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0724 21:45:37.213093 60452 deprecation_wrapper.py:119] From C:\Users\Dodo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3294: The name tf.log is deprecated. Please use tf.math.log instead.

W0724 21:45:37.221088 60452 deprecation.py:323] From C:\Users\Dodo\Anaconda3\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [23]:
mse_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
__________

In [24]:
bincross_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
__________

In [25]:
train_history=model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.25,
                        epochs=12, batch_size=128, verbose=1)   

Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 176s 5ms/step - loss: 2.1661 - acc: 0.1991 - val_loss: 2.0469 - val_acc: 0.2972
Epoch 2/12
37500/37500 [==============================] - 185s 5ms/step - loss: 1.9521 - acc: 0.2996 - val_loss: 1.9396 - val_acc: 0.3536
Epoch 3/12
37500/37500 [==============================] - 170s 5ms/step - loss: 1.8300 - acc: 0.3481 - val_loss: 1.8348 - val_acc: 0.3946
Epoch 4/12
37500/37500 [==============================] - 156s 4ms/step - loss: 1.7236 - acc: 0.3848 - val_loss: 1.7435 - val_acc: 0.4219
Epoch 5/12
37500/37500 [==============================] - 165s 4ms/step - loss: 1.6474 - acc: 0.4090 - val_loss: 1.6693 - val_acc: 0.4386
Epoch 6/12
37500/37500 [==============================] - 164s 4ms/step - loss: 1.5832 - acc: 0.4335 - val_loss: 1.6244 - val_acc: 0.4565
Epoch 7/12
37500/37500 [==============================] - 166s 4ms/step - loss: 1.5338 - acc: 0.4514 - val_loss: 1.5748 - 

In [ ]:
mse_train_history=mse_model.fit(x_img_train_normalize, y_label_train_OneHot,
                                validation_split=0.25,
                                epochs=12, batch_size=128, verbose=1)   

Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 183s 5ms/step - loss: 0.0909 - acc: 0.1020 - val_loss: 0.0899 - val_acc: 0.1270
Epoch 2/12
37500/37500 [==============================] - 190s 5ms/step - loss: 0.0903 - acc: 0.1077 - val_loss: 0.0897 - val_acc: 0.1352
Epoch 3/12
37500/37500 [==============================] - 171s 5ms/step - loss: 0.0901 - acc: 0.1146 - val_loss: 0.0896 - val_acc: 0.1462
Epoch 4/12
37500/37500 [==============================] - 191s 5ms/step - loss: 0.0899 - acc: 0.1236 - val_loss: 0.0895 - val_acc: 0.1638
Epoch 5/12
37500/37500 [==============================] - 163s 4ms/step - loss: 0.0897 - acc: 0.1282 - val_loss: 0.0894 - val_acc: 0.1753
Epoch 6/12
37500/37500 [==============================] - 164s 4ms/step - loss: 0.0895 - acc: 0.1389 - val_loss: 0.0893 - val_acc: 0.1817
Epoch 7/12
37500/37500 [==============================] - 161s 4ms/step - loss: 0.0893 - acc: 0.1429 - val_loss: 0.0892 - 

In [ ]:
bin_train_history=bincross_model.fit(x_img_train_normalize, y_label_train_OneHot,
                                     validation_split=0.25,
                                     epochs=12, batch_size=128, verbose=1)   

Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 173s 5ms/step - loss: 0.3247 - acc: 0.9000 - val_loss: 0.3226 - val_acc: 0.9000
Epoch 2/12
37500/37500 [==============================] - 166s 4ms/step - loss: 0.3212 - acc: 0.9000 - val_loss: 0.3206 - val_acc: 0.9000
Epoch 3/12
37500/37500 [==============================] - 165s 4ms/step - loss: 0.3182 - acc: 0.9000 - val_loss: 0.3180 - val_acc: 0.9000
Epoch 4/12
37500/37500 [==============================] - 192s 5ms/step - loss: 0.3137 - acc: 0.9000 - val_loss: 0.3140 - val_acc: 0.9000
Epoch 5/12
37500/37500 [==============================] - 164s 4ms/step - loss: 0.3080 - acc: 0.9000 - val_loss: 0.3091 - val_acc: 0.9000
Epoch 6/12
37500/37500 [==============================] - 161s 4ms/step - loss: 0.3023 - acc: 0.9000 - val_loss: 0.3043 - val_acc: 0.9000
Epoch 7/12
37500/37500 [==============================] - 169s 5ms/step - loss: 0.2974 - acc: 0.9002 - val_loss: 0.3004 - 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

#定義一個繪圖函數
def show_train_history(train_acc,test_acc, hist, title):
    plt.plot(hist.history[train_acc])
    plt.plot(hist.history[test_acc])
    plt.title(title)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train_acc', 'val_acc'], loc='upper left')
    plt.show()

In [ ]:
show_train_history('acc','val_acc', train_history, 'categorical_crossentropy')

In [ ]:
show_train_history('acc','val_acc', mse_train_history, 'MSE')

In [ ]:
show_train_history('acc','val_acc', bin_train_history, 'binary_crossentropy')